### CUSTOMIZE DATA PIPELINES
#### Design of Data pipelines
- 다수의 워커(프로세서)로 데이터를 로딩 시 Dataset, DataLoader를 사용한다.
- Dataset은 모델의 forward 메소드 실행에 필요한 Arguments에 대응하는 items을 가지는 딕셔너리를 리턴한다.
  (Dataset의 리턴 값은 모델 forward 메소드의 인자를 키 값으로 가지는 딕셔너리이다)
- Object Detection의 데이터는 모두 동일한 사이즈가 아닐 것이기 때문에, DataContainter 타입을 제안하여 각기 다른 사이즈의 데이터를 수집하고 분산 시키는데 사용한다.

- dataset 은 어노테이션을 어떻게 처리할 것인지를 정의하고, data pipeline은 data dict를 준비하기 위한 모든 스텝을 정의한다.
- 데이터 파이프라인은 연산의 연속으로 구성되는데, 각 연산은 (데이터) 딕셔너리를 입력으로 받아 (다음 Transform 입력을 위해) 딕셔너리를 넘겨준다.

![image](https://mmdetection3d.readthedocs.io/en/latest/_images/data_pipeline.png)

In [ ]:
train_pipeline = [
    dict(
        type='LoadPointsFromFile',
        load_dim=5,
        use_dim=5,
        backend_args=backend_args
    ),
    dict(
        type='LoadPointsFromMultiSweeps',
        sweeps_num=10,
        backend_args=backend_args
    ),
    dict(
        type='LoadAnnotations3D', 
        with_bbox_3d=True, 
        with_label_3d=True
    ),
    dict(
        type='GlobalRotScaleTrans',
        rot_range=[-0.3925, 0.3925],
        scale_ratio_range=[0.95, 1.05],
        translation_std=[0, 0, 0]
    ),
    dict(type='RandomFlip3D', flip_ratio_bev_horizontal=0.5),
    dict(type='PointsRangeFilter', point_cloud_range=point_cloud_range),
    dict(type='ObjectRangeFilter', point_cloud_range=point_cloud_range),
    dict(type='ObjectNameFilter', classes=class_names),
    dict(type='PointShuffle'),
    dict(type='DefaultFormatBundle3D', class_names=class_names),
    dict(type='Collect3D', keys=['points', 'gt_bboxes_3d', 'gt_labels_3d'])
]

test_pipeline = [
    dict(
        type='LoadPointsFromFile',
        load_dim=5,
        use_dim=5,
        backend_args=backend_args),
    dict(
        type='LoadPointsFromMultiSweeps',
        sweeps_num=10,
        backend_args=backend_args),
    dict(
        type='MultiScaleFlipAug',
        img_scale=(1333, 800),
        pts_scale_ratio=1.0,
        flip=False,
        pcd_horizontal_flip=False,
        pcd_vertical_flip=False,
        transforms=[
            dict(
                type='GlobalRotScaleTrans',
                rot_range=[0, 0],
                scale_ratio_range=[1., 1.],
                translation_std=[0, 0, 0]),
            dict(type='RandomFlip3D'),
            dict(
                type='PointsRangeFilter', point_cloud_range=point_cloud_range),
            dict(
                type='DefaultFormatBundle3D',
                class_names=class_names,
                with_label=False),
            dict(type='Collect3D', keys=['points'])
        ]
    )
]    

DataLoading  
- LoadPointsFromFile -> add: points  
- LoadPointsFromMultiSweeps -> update: points  
- LoadAnnotations3D --> add: gt_bboxes_3d, gt_labels_3d, gt_bboxes, gt_labels, pts_instance_mask, pts_semantic_mask, bbox3d_fields, pts_mask_fields, pts_seg_fields  
  
Pre-processing  
- GlobalRotScaleTrans  
  -> add:pcd_trans, pcd_rotation, pcd_scale_factor   
  -> update: points, *bbox3d_fields  
- RandomFlip3D  
  -> add: flip, pcd_horizontal_flip, pcd_vertical_flip  
  -> update: points, *bbox3d_fields  
- PointsRangeFilter -> update: points  
- ObjectRangeFilter -> update: gt_bboxes_3d, gt_labels_3d  
- ObjectNameFilter -> update: gt_bboxes_3d, gt_labels_3d  
- PointShuffle -> update: points  
- PointsRangeFilter -> update: points  
  
Formatting
- DefaultFormatBundle3D  
 -> update: points, gt_bboxes_3d, gt_labels_3d, gt_bboxes, gt_labels  
- Collect3D  
  -> add: img_meta (the keys of img_meta is specified by meta_keys)  
  -> remove: all other keys except for those specified by keys  
  
Test Time Augmentation
- MultiScaleFlipAug
  -> scale, pcd_scale_factor, flip, flip_direction, pcd_horizontal_flip, pcd_vertical_flip

(해당 부분이 중요한 부분이라면, 나중에 표로 정리해도 좋을 듯 하다. Augmentation 옵션 별(row) / add, update 여부(column) )

### Extend and use custom pipelines
1. 가칭 my_pipeline.py 은 Data Pipeline과 마찬가지로 dict을 입, 출력으로 가진다.

In [ ]:
from mmdet.datasets import PIPELINES

@PIPELINES.register_module()
class MyTransform:
    def __call__(self, results):
        results['dummy'] = True
        return results

2. 새로운 클래스를 임포트 한다.

In [ ]:
train_pipeline = [
    dict(
        type='LoadPointsFromFile',
        load_dim=5,
        use_dim=5,
        backend_args=backend_args
    ),
    dict(
        type='LoadPointsFromMultiSweeps',
        sweeps_num=10,
        backend_args=backend_args
    ),
    dict(
        type='LoadAnnotations3D', 
        with_bbox_3d=True, 
        with_label_3d=True
    ),
    dict(
        type='GlobalRotScaleTrans',
        rot_range=[-0.3925, 0.3925],
        scale_ratio_range=[0.95, 1.05],
        translation_std=[0, 0, 0]
    ),
    dict(type='RandomFlip3D', flip_ratio_bev_horizontal=0.5),
    dict(type='PointsRangeFilter', point_cloud_range=point_cloud_range),
    dict(type='ObjectRangeFilter', point_cloud_range=point_cloud_range),
    dict(type='ObjectNameFilter', classes=class_names),
    dict(type='MyTransform'), # <--------------------------------------------------- Here!
    dict(type='PointShuffle'),
    dict(type='DefaultFormatBundle3D', class_names=class_names),
    dict(type='Collect3D', keys=['points', 'gt_bboxes_3d', 'gt_labels_3d'])
]

※ DataContainer
- 텐서는 collate function에 쌓이게(stacked) 되고, 동일 차원의 값으로 슬라이스 한다. 
- 이때 모든 텐서는 동일사이즈이며, 타입이 (numpy array or Tensor)로 제한되게 된다.
- 이 문제를 해결하기 위해 DataContainer and MMDataParallel 을 도입하였다.

https://mmdetection3d.readthedocs.io/en/latest/_images/data_pipeline.png

In [ ]:
class DataContainer:
    """A container for any type of objects.

    Typically tensors will be stacked in the collate function and sliced along
    some dimension in the scatter function. This behavior has some limitations.
    1. All tensors have to be the same size.
    2. Types are limited (numpy array or Tensor).

    We design `DataContainer` and `MMDataParallel` to overcome these
    limitations. The behavior can be either of the following.

    - copy to GPU, pad all tensors to the same size and stack them
    - copy to GPU without stacking
    - leave the objects as is and pass it to the model
    - pad_dims specifies the number of last few dimensions to do padding
    """

    def __init__(self,
                data: Union[torch.Tensor, np.ndarray],
                stack: bool = False,
                padding_value: int = 0,
                cpu_only: bool = False,
                pad_dims: int = 2):
        self._data = data
        self._cpu_only = cpu_only
        self._stack = stack
        self._padding_value = padding_value
        assert pad_dims in [None, 1, 2, 3]
        self._pad_dims = pad_dims

    